In [269]:
! pip install catboost

In [270]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
# from pandas_profiling import ProfileReport

#model
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier #boosted decision tree
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
# from sklearn.ensemble import VotingClassifier

#scoring
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import missingno as msno

import warnings
warnings.filterwarnings("ignore")

In [271]:
# df = pd.read_csv("df_train2.csv")
# df_stand = pd.read_csv("df_train_stand2.csv")
# df_val = pd.read_csv("df_val2.csv")


df = pd.read_csv("https://raw.githubusercontent.com/BryanT05/Machine-Learning-Bootcamp/main/Final%20Project/Preprocess%20Data/df_train.csv")
df_stand = pd.read_csv("https://raw.githubusercontent.com/BryanT05/Machine-Learning-Bootcamp/main/Final%20Project/Preprocess%20Data/df_train_stand.csv")

df_val = pd.read_csv("https://raw.githubusercontent.com/BryanT05/Machine-Learning-Bootcamp/main/Final%20Project/Preprocess%20Data/df_val.csv")

In [272]:
X = df.drop(['stroke'], axis = 1)
y = df['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 42, stratify = y)

X_stand = df_stand.drop(['stroke'], axis = 1)
y_stand = df_stand['stroke']
X_stand_train, X_stand_test, y_stand_train, y_stand_test = train_test_split(X_stand, y_stand, test_size = 0.2,random_state = 42, stratify = y_stand)

X_val = df_val.drop(['stroke'], axis = 1)
y_val = df_val['stroke']

In [273]:
y_train.value_counts()

1    3111
0    3110
Name: stroke, dtype: int64

In [274]:
y_test.value_counts()

1    778
0    778
Name: stroke, dtype: int64

#Baseline Model

In [275]:
model = DummyClassifier(strategy='constant', constant = 1, random_state= 42)
scores = cross_val_score(model, X, y, scoring='roc_auc', n_jobs=-1)
print('Mean roc_auc: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Mean roc_auc: 0.500 (0.000)


without standarization

In [276]:
model = LogisticRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76       778
           1       0.75      0.81      0.78       778

    accuracy                           0.77      1556
   macro avg       0.77      0.77      0.77      1556
weighted avg       0.77      0.77      0.77      1556



With standarization

In [277]:
model = LogisticRegression().fit(X_stand_train, y_stand_train)
y_pred = model.predict(X_stand_test)
print(classification_report(y_stand_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76       778
           1       0.75      0.81      0.78       778

    accuracy                           0.77      1556
   macro avg       0.77      0.77      0.77      1556
weighted avg       0.77      0.77      0.77      1556



In [278]:
roc_auc_score(y_test, y_pred)

0.768637532133676

In [279]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6221 entries, 6525 to 5288
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             6221 non-null   float64
 1   hypertension                    6221 non-null   float64
 2   heart_disease                   6221 non-null   float64
 3   avg_glucose_level               6221 non-null   float64
 4   bmi                             6221 non-null   float64
 5   gender_Female                   6221 non-null   float64
 6   gender_Male                     6221 non-null   float64
 7   ever_married_No                 6221 non-null   float64
 8   ever_married_Yes                6221 non-null   float64
 9   work_type_Govt_job              6221 non-null   float64
 10  work_type_Never_worked          6221 non-null   float64
 11  work_type_Private               6221 non-null   float64
 12  work_type_Self-employed        

In [280]:
clf = CatBoostClassifier(iterations=100)
clf.fit( X_train, y_train)

Learning rate set to 0.185743
0:	learn: 0.5251103	total: 9.13ms	remaining: 904ms
1:	learn: 0.4142646	total: 17.5ms	remaining: 858ms
2:	learn: 0.3117341	total: 26.8ms	remaining: 867ms
3:	learn: 0.2663158	total: 37ms	remaining: 887ms
4:	learn: 0.2277078	total: 45.8ms	remaining: 870ms
5:	learn: 0.2065185	total: 53.8ms	remaining: 842ms
6:	learn: 0.1829677	total: 62.4ms	remaining: 829ms
7:	learn: 0.1701164	total: 70.6ms	remaining: 812ms
8:	learn: 0.1621507	total: 78.3ms	remaining: 791ms
9:	learn: 0.1473276	total: 86.2ms	remaining: 776ms
10:	learn: 0.1406086	total: 93.9ms	remaining: 760ms
11:	learn: 0.1356941	total: 107ms	remaining: 782ms
12:	learn: 0.1326093	total: 115ms	remaining: 769ms
13:	learn: 0.1270667	total: 123ms	remaining: 753ms
14:	learn: 0.1237633	total: 134ms	remaining: 759ms
15:	learn: 0.1195474	total: 142ms	remaining: 744ms
16:	learn: 0.1170865	total: 149ms	remaining: 730ms
17:	learn: 0.1148084	total: 160ms	remaining: 730ms
18:	learn: 0.1130053	total: 169ms	remaining: 719ms
19

In [281]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       778
           1       0.99      0.96      0.97       778

    accuracy                           0.97      1556
   macro avg       0.98      0.97      0.97      1556
weighted avg       0.98      0.97      0.97      1556



#Model Selection

In [282]:
models = []
names = []

models.append(LogisticRegression(solver='liblinear'))
names.append('LogisticRegression')

models.append(GaussianNB())
names.append('Naive Bayes')

models.append(KNeighborsClassifier())
names.append('KNN')

models.append(SVC(gamma='scale', random_state = 42))
names.append('SVM')

models.append(DecisionTreeClassifier(random_state = 42))
names.append('Decision Tree')

models.append(BaggingClassifier(random_state = 42))
names.append('Bagging Decision Tree')

models.append(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=25, learning_rate = 0.1, random_state = 42))
names.append('Boosted Decision tree')

models.append(RandomForestClassifier(random_state = 42))
names.append('Random Forest')

models.append(CatBoostClassifier(iterations=100, random_state = 42))
names.append('CatBoost Classifier')

In [283]:
def model_score(model, name):
  scores = cross_val_score(model, X, y, scoring='roc_auc', n_jobs=-1)
  print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
  # print(f'{name} {np.mean(scores)} {np.std(scores)}')

def model_score_stand(model, name):
  scores = cross_val_score(model, X_stand, y_stand, scoring='roc_auc', n_jobs=-1)
  print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))


def model_score2(model, name):
  y_pred = model.fit(X_train, y_train).predict(X_test)
  try:
    score  = model.predict_proba(X_test)[:, 1]
    roc = roc_auc_score(y_test, score, average ='weighted')
  except:
    roc = 0
  print(f'{name}:')
  print(f'ROC score {roc}')
  print(f'Accuracy {accuracy_score(y_test,y_pred)}')
  print(f'f1 {f1_score(y_test,y_pred, average = "weighted")}')
  print(f'Recall {recall_score(y_test,y_pred, average = "weighted")}')
  print(f'Precision {precision_score(y_test,y_pred, average = "weighted")}')
  print()

def model_score2_stand(model, name):
  y_pred = model.fit(X_stand_train, y_stand_train).predict(X_stand_test)
  try:
    score  = model.predict_proba(X_stand_test)[:, 1]
    roc = roc_auc_score(y_stand_test, score, average ='weighted')
  except:
    roc = 0
  print(f'{name}:')
  print(f'ROC score {roc}')
  print(f'Accuracy {accuracy_score(y_stand_test,y_pred)}')
  print(f'f1 {f1_score(y_stand_test,y_pred, average = "weighted")}')
  print(f'Recall {recall_score(y_stand_test,y_pred, average = "weighted")}')
  print(f'Precision {precision_score(y_stand_test,y_pred, average = "weighted")}')
  print()

##Without standarization

In [284]:
for i in range(len(models)):
  model_score(models[i],names[i])

>LogisticRegression 0.854 (0.005)
>Naive Bayes 0.830 (0.010)
>KNN 0.942 (0.005)
>SVM 0.848 (0.007)
>Decision Tree 0.941 (0.001)
>Bagging Decision Tree 0.989 (0.001)
>Boosted Decision tree 0.940 (0.002)
>Random Forest 0.993 (0.001)
>CatBoost Classifier 0.992 (0.002)


In [285]:
for i in range(len(models)):
  model_score2(models[i],names[i])

LogisticRegression:
ROC score 0.8378562790359566
Accuracy 0.769280205655527
f1 0.7689250707884249
Recall 0.769280205655527
Precision 0.7709458527203301

Naive Bayes:
ROC score 0.8263096331639364
Accuracy 0.6838046272493573
f1 0.6565941114936938
Recall 0.6838046272493573
Precision 0.7690934597523219

KNN:
ROC score 0.9516152748131456
Accuracy 0.8862467866323908
f1 0.8851066971867386
Recall 0.8862467866323908
Precision 0.9022114066190804

SVM:
ROC score 0
Accuracy 0.7570694087403599
f1 0.7563928608704727
Recall 0.7570694087403599
Precision 0.759957230686982

Decision Tree:
ROC score 0.9357326478149101
Accuracy 0.9357326478149101
f1 0.9357299932756061
Recall 0.9357326478149101
Precision 0.9358046478426397

Bagging Decision Tree:
ROC score 0.9910546123803041
Accuracy 0.9640102827763496
f1 0.9640030867716631
Recall 0.9640102827763496
Precision 0.9643816137566138

Boosted Decision tree:
ROC score 0.9376606683804627
Accuracy 0.9376606683804627
f1 0.9376548745492916
Recall 0.9376606683804627
P

##With standarization

In [286]:
for i in range(len(models)):
  model_score_stand(models[i],names[i])

>LogisticRegression 0.854 (0.005)
>Naive Bayes 0.830 (0.010)
>KNN 0.973 (0.006)
>SVM 0.982 (0.002)
>Decision Tree 0.941 (0.001)
>Bagging Decision Tree 0.989 (0.001)
>Boosted Decision tree 0.940 (0.002)
>Random Forest 0.993 (0.001)
>CatBoost Classifier 0.992 (0.002)


In [287]:
for i in range(len(models)):
  model_score2_stand(models[i],names[i])

LogisticRegression:
ROC score 0.8374977696420194
Accuracy 0.7686375321336761
f1 0.7682930057711654
Recall 0.7686375321336761
Precision 0.7702448461318566

Naive Bayes:
ROC score 0.8263079810469136
Accuracy 0.6574550128534704
f1 0.6126885833105667
Recall 0.6574550128534704
Precision 0.7928469808845174

KNN:
ROC score 0.9697902141804509
Accuracy 0.9158097686375322
f1 0.9156030932338306
Recall 0.9158097686375322
Precision 0.9199230839819473

SVM:
ROC score 0
Accuracy 0.9338046272493573
f1 0.9337586357258142
Recall 0.9338046272493573
Precision 0.9350127484455842

Decision Tree:
ROC score 0.9357326478149101
Accuracy 0.9357326478149101
f1 0.9357299932756061
Recall 0.9357326478149101
Precision 0.9358046478426397

Bagging Decision Tree:
ROC score 0.9910546123803041
Accuracy 0.9640102827763496
f1 0.9640030867716631
Recall 0.9640102827763496
Precision 0.9643816137566138

Boosted Decision tree:
ROC score 0.9376606683804627
Accuracy 0.9376606683804627
f1 0.9376548745492916
Recall 0.937660668380462

since Random Forest and Catboost classifier is the models with best score we will compare this two models for unseen data.

since there is no difference for random forest and catboost classifier whether the data is standarize or not, we will use the non standarize data

#Hyperparameter tuning

In [288]:
# param_grid_RF = {'bootstrap': [True, False],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

# rf = RandomForestClassifier(random_state = 42)
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid_RF, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X, y)

In [289]:
# rf_best_params = rf_random.best_params_

In [290]:
# rf_best_params

In [291]:
RF = RandomForestClassifier(bootstrap= False,
 max_depth= 100,
 max_features= 'auto',
 min_samples_leaf= 1,
 min_samples_split= 10,
 n_estimators= 600)

In [292]:
# catboost_params = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}
# CB = CatBoostClassifier(random_state = 42)
# gscv = GridSearchCV(estimator = CB, param_grid = catboost_params, cv = 3, verbose=2, n_jobs = -1)
# gscv.fit(X, y)

In [293]:
# cb_best_params = gscv.best_params_

In [294]:
# cb_best_params

In [295]:
CBC = CatBoostClassifier(max_depth= 3, n_estimators= 200)

#final score

###With hyperparameter Tuning

In [296]:
def final_score(model):
  y_pred = model.fit(X, y).predict(X_val)
  try:
    score  = model.predict_proba(X_val)[:, 1]
    roc = roc_auc_score(y_val, score, average ='weighted')
  except:
    roc = 0
  print(f'ROC score {roc}')
  print(f'Accuracy {accuracy_score(y_val,y_pred)}')
  print(f'f1 {f1_score(y_val,y_pred, average = "weighted")}')
  print(f'Recall {recall_score(y_val,y_pred, average = "weighted")}')
  print(f'Precision {precision_score(y_val,y_pred, average = "weighted")}')
  print()

def class_report(model):
  y_pred = model.fit(X, y).predict(X_val)
  print(classification_report(y_val, y_pred))

In [297]:
final_score(RF)

ROC score 0.9952968841857731
Accuracy 0.9763496143958869
f1 0.9763421724479074
Recall 0.9763496143958869
Precision 0.9769321656391948



In [298]:
final_score(CBC)

Learning rate set to 0.10821
0:	learn: 0.6316854	total: 4.42ms	remaining: 880ms
1:	learn: 0.5595767	total: 9ms	remaining: 891ms
2:	learn: 0.4919479	total: 13.2ms	remaining: 867ms
3:	learn: 0.4346327	total: 17.5ms	remaining: 857ms
4:	learn: 0.3928915	total: 21.7ms	remaining: 846ms
5:	learn: 0.3682458	total: 26ms	remaining: 840ms
6:	learn: 0.3250987	total: 30.2ms	remaining: 834ms
7:	learn: 0.2961341	total: 34.6ms	remaining: 831ms
8:	learn: 0.2731360	total: 38.9ms	remaining: 827ms
9:	learn: 0.2615991	total: 43.3ms	remaining: 823ms
10:	learn: 0.2517416	total: 47.7ms	remaining: 819ms
11:	learn: 0.2383065	total: 52.2ms	remaining: 818ms
12:	learn: 0.2269338	total: 58ms	remaining: 835ms
13:	learn: 0.2181925	total: 65ms	remaining: 863ms
14:	learn: 0.2066817	total: 69.3ms	remaining: 855ms
15:	learn: 0.1988039	total: 73.5ms	remaining: 846ms
16:	learn: 0.1894366	total: 77.8ms	remaining: 838ms
17:	learn: 0.1868540	total: 82ms	remaining: 829ms
18:	learn: 0.1804522	total: 86.2ms	remaining: 821ms
19:	

In [299]:
class_report(RF)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       973
           1       1.00      0.96      0.98       972

    accuracy                           0.98      1945
   macro avg       0.98      0.98      0.98      1945
weighted avg       0.98      0.98      0.98      1945



In [300]:
class_report(CBC)

Learning rate set to 0.10821
0:	learn: 0.6316854	total: 4.04ms	remaining: 805ms
1:	learn: 0.5595767	total: 8.74ms	remaining: 865ms
2:	learn: 0.4919479	total: 13.1ms	remaining: 863ms
3:	learn: 0.4346327	total: 17.3ms	remaining: 849ms
4:	learn: 0.3928915	total: 21.5ms	remaining: 840ms
5:	learn: 0.3682458	total: 27.5ms	remaining: 889ms
6:	learn: 0.3250987	total: 32.2ms	remaining: 887ms
7:	learn: 0.2961341	total: 36.5ms	remaining: 876ms
8:	learn: 0.2731360	total: 40.9ms	remaining: 867ms
9:	learn: 0.2615991	total: 45.1ms	remaining: 857ms
10:	learn: 0.2517416	total: 49.5ms	remaining: 850ms
11:	learn: 0.2383065	total: 53.8ms	remaining: 843ms
12:	learn: 0.2269338	total: 58.1ms	remaining: 836ms
13:	learn: 0.2181925	total: 62.4ms	remaining: 829ms
14:	learn: 0.2066817	total: 66.7ms	remaining: 823ms
15:	learn: 0.1988039	total: 71.2ms	remaining: 818ms
16:	learn: 0.1894366	total: 75.5ms	remaining: 813ms
17:	learn: 0.1868540	total: 79.8ms	remaining: 807ms
18:	learn: 0.1804522	total: 84.1ms	remaining:

From the score we can see RandomForest model perform better so we will save RandomForest Model

###Without hyperparameter tuning

In [301]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(RF, open(filename, 'wb'))

In [302]:
loaded_model = pickle.load(open(filename, 'rb'))

In [303]:
final_score(loaded_model)

ROC score 0.9955538214930701
Accuracy 0.9763496143958869
f1 0.9763421724479074
Recall 0.9763496143958869
Precision 0.9769321656391948



In [304]:
class_report(loaded_model)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       973
           1       1.00      0.96      0.98       972

    accuracy                           0.98      1945
   macro avg       0.98      0.98      0.98      1945
weighted avg       0.98      0.98      0.98      1945

